In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel
import tracemalloc
tracemalloc.start()

In [52]:
rec_system = pd.read_parquet("../datasets/rec_System.parquet")#.head(10000)
rec_system_copy = rec_system.copy()

In [53]:
rec_system.shape

(42236, 11)

In [51]:
rec_system.shape

(9980, 12)

In [4]:
# rec_system[rec_system["title"]=="The Toy"]
rec_system.drop_duplicates(subset="title",inplace=True)

In [5]:
rec_system["release_year"] = rec_system["release_year"].astype(str)
rec_system["popularity"] = rec_system["popularity"].astype(str)

In [6]:
rec_system.fillna({"overview":"[Unknown]",
                   "name_genre":"[Unknown]",
                   "actors_names":"[Unknown]",
                   "director_names":"[Unknown]",
                   "tagline":"[Unknown]",
                   "company":"Unknown",
                   "release_year": rec_system["release_year"].mode(),
                   "popularity": "0"
                   },inplace=True)

In [7]:
rec_system["overview"].iloc[0][0:100]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto "

* Transformo la columna overview a una lista con la funcion `split`

In [8]:
#NO
# rec_system["overview"] = rec_system["overview"].apply(lambda x: x.split())
# rec_system["name_genre"] = rec_system["name_genre"].apply(lambda x: x.split())
# rec_system["tagline"] = rec_system["tagline"].apply(lambda x: x.split())

In [9]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,company,collection,release_year,vote_average,popularity
0,toy story,"Led by Woody, Andy's toys live happily in his ...","[Animation, Family, Comedy]","[Erik von Detten, Jim Varney, Wallace Shawn, P...",[John Lasseter],[Unknown],Pixar Animation Studios,Toy Story Collection,1995,7.7,21.946943
1,jumanji,When siblings Judy and Peter discover an encha...,"[Family, Adventure, Fantasy]","[Bonnie Hunt, Brandon Obray, Bradley Pierce, G...",[Joe Johnston],Roll the dice and unleash the excitement!,TriStar Pictures Teitler Film Interscope Commu...,[],1995,6.9,17.015539
2,grumpier old men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[Walter Matthau, Ann-Margret, Jack Lemmon, Bur...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,Warner Bros. Lancaster Gate,Grumpy Old Men Collection,1995,6.5,11.7129
3,waiting to exhale,"Cheated on, mistreated and stepped on, the wom...","[Drama, Romance, Comedy]","[Michael Beach, Angela Bassett, Loretta Devine...",[Forest Whitaker],Friends are the people who let you be yourself...,Twentieth Century Fox Film Corporation,[],1995,6.1,3.859495
4,father of the bride part ii,Just when George Banks has recovered from his ...,[Comedy],"[Lori Alan, Kieran Culkin, Diane Keaton, Peter...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,Sandollar Productions Touchstone Pictures,Father of the Bride Collection,1995,5.7,8.387519
...,...,...,...,...,...,...,...,...,...,...,...
9995,race with the devil,For old friends Roger (Peter Fonda) and Frank ...,"[Horror, Action, Thriller]","[Phil Hoover, Wes Bishop, Warren Oates, Paul A...",[Jack Starrett],"If you're going to race with the devil, you've...",Twentieth Century Fox Film Corporation Saber P...,[],1975,6.9,2.621146
9996,acacia,A Korean horror film about an adopted young bo...,[Horror],"[Shim Hye-Jin, Jung Hee-tae, Jung Na-Yoon, Kim...",[Ki-hyeong Park],[Unknown],Show East,[],2003,5.4,1.46791
9997,mermaid chronicles part 1 she creature,Two carnies (Sewell and Gugino) abduct a merma...,[Horror],"[Rufus Sewell, Reno Wilson, Brian Sieve, Dan H...",[Sebastian Gutierrez],"Beatiful, seductive and totally deadly.",Columbia TriStar Home Box Office (HBO) Creatur...,[],2001,5.8,2.088049
9998,attila,"A romanced story of Attila the Hun, since his ...","[History, Action, War, Drama, Adventure, Romance]","[Siân Phillips, Tim Curry, Finbar Lynch, David...",[Dick Lowry],Men followed. Women worshiped. Rome trembled.,Alphaville Films USA Cable Network Attila Prod...,[],2001,6.5,9.563499


In [10]:
def collapse(valor):
    valores =[]
    for i in valor:
       valores.append(i)
    return valores

In [11]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,company,collection,release_year,vote_average,popularity
0,toy story,"Led by Woody, Andy's toys live happily in his ...","[Animation, Family, Comedy]","[Erik von Detten, Jim Varney, Wallace Shawn, P...",[John Lasseter],[Unknown],Pixar Animation Studios,Toy Story Collection,1995,7.7,21.946943
1,jumanji,When siblings Judy and Peter discover an encha...,"[Family, Adventure, Fantasy]","[Bonnie Hunt, Brandon Obray, Bradley Pierce, G...",[Joe Johnston],Roll the dice and unleash the excitement!,TriStar Pictures Teitler Film Interscope Commu...,[],1995,6.9,17.015539
2,grumpier old men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[Walter Matthau, Ann-Margret, Jack Lemmon, Bur...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,Warner Bros. Lancaster Gate,Grumpy Old Men Collection,1995,6.5,11.7129
3,waiting to exhale,"Cheated on, mistreated and stepped on, the wom...","[Drama, Romance, Comedy]","[Michael Beach, Angela Bassett, Loretta Devine...",[Forest Whitaker],Friends are the people who let you be yourself...,Twentieth Century Fox Film Corporation,[],1995,6.1,3.859495
4,father of the bride part ii,Just when George Banks has recovered from his ...,[Comedy],"[Lori Alan, Kieran Culkin, Diane Keaton, Peter...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,Sandollar Productions Touchstone Pictures,Father of the Bride Collection,1995,5.7,8.387519
...,...,...,...,...,...,...,...,...,...,...,...
9995,race with the devil,For old friends Roger (Peter Fonda) and Frank ...,"[Horror, Action, Thriller]","[Phil Hoover, Wes Bishop, Warren Oates, Paul A...",[Jack Starrett],"If you're going to race with the devil, you've...",Twentieth Century Fox Film Corporation Saber P...,[],1975,6.9,2.621146
9996,acacia,A Korean horror film about an adopted young bo...,[Horror],"[Shim Hye-Jin, Jung Hee-tae, Jung Na-Yoon, Kim...",[Ki-hyeong Park],[Unknown],Show East,[],2003,5.4,1.46791
9997,mermaid chronicles part 1 she creature,Two carnies (Sewell and Gugino) abduct a merma...,[Horror],"[Rufus Sewell, Reno Wilson, Brian Sieve, Dan H...",[Sebastian Gutierrez],"Beatiful, seductive and totally deadly.",Columbia TriStar Home Box Office (HBO) Creatur...,[],2001,5.8,2.088049
9998,attila,"A romanced story of Attila the Hun, since his ...","[History, Action, War, Drama, Adventure, Romance]","[Siân Phillips, Tim Curry, Finbar Lynch, David...",[Dick Lowry],Men followed. Women worshiped. Rome trembled.,Alphaville Films USA Cable Network Attila Prod...,[],2001,6.5,9.563499


In [12]:
# Con parquet, los tipos list pasan a ser ndarray, con este metodo los volvemos lista
rec_system["name_genre"]=rec_system["name_genre"].apply(collapse)
rec_system["actors_names"]=rec_system["actors_names"].apply(collapse)
# rec_system["director_names"]=rec_system["director_names"].apply(collapse)
# rec_system["company"]=rec_system["company"].apply(collapse)
# rec_system["tagline"]=rec_system["tagline"].apply(collapse)

In [13]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,company,collection,release_year,vote_average,popularity
0,toy story,"Led by Woody, Andy's toys live happily in his ...","[Animation, Family, Comedy]","[Erik von Detten, Jim Varney, Wallace Shawn, P...",[John Lasseter],[Unknown],Pixar Animation Studios,Toy Story Collection,1995,7.7,21.946943
1,jumanji,When siblings Judy and Peter discover an encha...,"[Family, Adventure, Fantasy]","[Bonnie Hunt, Brandon Obray, Bradley Pierce, G...",[Joe Johnston],Roll the dice and unleash the excitement!,TriStar Pictures Teitler Film Interscope Commu...,[],1995,6.9,17.015539
2,grumpier old men,A family wedding reignites the ancient feud be...,"[Romance, Comedy]","[Walter Matthau, Ann-Margret, Jack Lemmon, Bur...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,Warner Bros. Lancaster Gate,Grumpy Old Men Collection,1995,6.5,11.7129
3,waiting to exhale,"Cheated on, mistreated and stepped on, the wom...","[Drama, Romance, Comedy]","[Michael Beach, Angela Bassett, Loretta Devine...",[Forest Whitaker],Friends are the people who let you be yourself...,Twentieth Century Fox Film Corporation,[],1995,6.1,3.859495
4,father of the bride part ii,Just when George Banks has recovered from his ...,[Comedy],"[Lori Alan, Kieran Culkin, Diane Keaton, Peter...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,Sandollar Productions Touchstone Pictures,Father of the Bride Collection,1995,5.7,8.387519
...,...,...,...,...,...,...,...,...,...,...,...
9995,race with the devil,For old friends Roger (Peter Fonda) and Frank ...,"[Horror, Action, Thriller]","[Phil Hoover, Wes Bishop, Warren Oates, Paul A...",[Jack Starrett],"If you're going to race with the devil, you've...",Twentieth Century Fox Film Corporation Saber P...,[],1975,6.9,2.621146
9996,acacia,A Korean horror film about an adopted young bo...,[Horror],"[Shim Hye-Jin, Jung Hee-tae, Jung Na-Yoon, Kim...",[Ki-hyeong Park],[Unknown],Show East,[],2003,5.4,1.46791
9997,mermaid chronicles part 1 she creature,Two carnies (Sewell and Gugino) abduct a merma...,[Horror],"[Rufus Sewell, Reno Wilson, Brian Sieve, Dan H...",[Sebastian Gutierrez],"Beatiful, seductive and totally deadly.",Columbia TriStar Home Box Office (HBO) Creatur...,[],2001,5.8,2.088049
9998,attila,"A romanced story of Attila the Hun, since his ...","[History, Action, War, Drama, Adventure, Romance]","[Siân Phillips, Tim Curry, Finbar Lynch, David...",[Dick Lowry],Men followed. Women worshiped. Rome trembled.,Alphaville Films USA Cable Network Attila Prod...,[],2001,6.5,9.563499


In [14]:
rec_system["vote_average"] = rec_system["vote_average"].astype(str)

In [15]:
rec_system["name_genre"] = rec_system["name_genre"].apply(lambda x: ",".join(x))
rec_system["actors_names"] = rec_system["actors_names"].apply(lambda x: ",".join(x))
rec_system["director_names"] = rec_system["director_names"].apply(lambda x: ",".join(x))

In [39]:
rec_system["tags"] = rec_system["name_genre"]  + " " + rec_system["vote_average"] + " "  + rec_system["popularity"] + " " + rec_system["company"] + " " + rec_system["release_year"]

In [40]:
# max_features: maxima cantidad de palabras a vectorizar
# cv = CountVectorizer(max_features=1000, stop_words="english")
tv = TfidfVectorizer(max_features=30000, stop_words="english")

In [41]:
vector = tv.fit_transform(rec_system["tags"]).toarray()

In [42]:
rec_system["tags"]=rec_system["tags"].str.replace(","," ")

In [43]:
# tv.vocabulary_ # Chequeo palabras que se repitieron

In [44]:
cosine_sim = linear_kernel(vector,vector)

In [45]:
# numeric_features = a.select_dtypes(include=[np.number]).columns.tolist()

In [46]:
indices = pd.Series(rec_system.index, index=rec_system["title"])
indices = indices.drop_duplicates()

In [49]:
def recommend(movie,cosine_sim=cosine_sim):
    movie = movie.lower()
    num_movies = 6
    try:
        index = indices[movie]
        sim_scores=list(enumerate(cosine_sim[index]))
        distances = sorted(sim_scores,reverse= True,key=lambda x: x[1])
        
        for i in distances[1:num_movies]:
            # print(i[0])
            titulo = rec_system.iloc[i[0]].title
            print(titulo.title())
    except:
        return "Titulo Desconocido"
recommend("toy story")

Toy Story 2
Finding Nemo
A Bugs Life
Monsters Inc
The Incredibles


In [48]:
print(tracemalloc.get_traced_memory())
tracemalloc.stop()

(0, 0)
